In [1]:
# data extraction and preprocessing
import pandas as pd
import numpy as np

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import warnings

warnings.filterwarnings('ignore')

# Read and preprocess the data
data = pd.read_csv(r"D:\mas\datasets\World bank\S0 RawWDI1_manual_tax_labor_no_lag.csv")
data = data.drop("Unnamed: 0", axis=1)
data.replace('..', np.nan, inplace=True)

# data.iloc[:, 2:] = data.iloc[:, 2:].replace('..', pd.NA)
numeric_col = data.columns[2:]
data[numeric_col] = data[numeric_col].apply(pd.to_numeric, errors='coerce')

"""
# Fill the missing value with country-specific mean
grouped_df = data.groupby("Country Name")
mean_values = grouped_df.transform('mean')

df_filled = data.fillna(mean_values)
df_filled.to_csv(r"D:\mas\datasets\World bank\RawWDI1_filled_na_with_mean.csv")
df_filled.head()
"""

# print(data.dtypes)
#data.info(verbose=True)
#data.iloc[120:135,:20]

#Jan 30, 2025
"""
Basic steps to deal with missing values
s1: deal with the file with all 7 variables without lag terms
"D:\mas\datasets\World bank\S0 RawWDI1_manual_tax_labor_no_lag.csv"
using the following code.

s2: generate the lag terms of all 7 variables from 1961 to 2021

s3: deal with the missing values in lag terms of all 7 variables in the year of 1960
"""


# step 1
num_cols = data.columns[2:9]
data[num_cols] = data[num_cols].apply(pd.to_numeric, errors='coerce') # Convert to float

'''
Since GDP per capita generally follows a gradual increasing trend, 
we use linear interpolation within each country.
'''
data['GDP pc'] = data.groupby('Country Name')['GDP pc'].apply(lambda x: x.interpolate(method='linear')).fillna(method='bfill').fillna(method='ffill')
data['ln_GDPpc'] = data.groupby('Country Name')['ln_GDPpc'].apply(lambda x: x.interpolate(method='linear')).fillna(method='bfill').fillna(method='ffill')

'''
Since the rest variables other than GDP do not follow a clear trend, 
use country-wise mean imputation
'''
numeric_cols = data.columns[4:9]
#data[numeric_cols] = data.groupby('Country Name')[numeric_cols].transform\
#(lambda x: x.fillna(x.mean()))

data[numeric_cols] = data.groupby('Country Name')[numeric_cols].transform(lambda x: x.fillna(x.mean()))

# Feb 6, 2025
# step 2
#data[data.index % 62 == 0].iloc[1:]
col_names = list(num_cols)
for col in col_names:
    data[f'lag {col}'] = data.groupby('Country Name')[col].shift(1)

data.drop('Unnamed: 72', axis=1, inplace=True)
reorder_col_names = (data.columns[:9].tolist() + data.columns[-7:].tolist() + data.columns[9:-7].tolist())
data = data[reorder_col_names]

# step 3
data.fillna(method='bfill',inplace=True)
data.iloc[60:64, :17]


,Country Name,Year,GDP pc,ln_GDPpc,tax,unemp,age_dr,labor,gdp_deflator,lag GDP pc,lag ln_GDPpc,lag tax,lag unemp,lag age_dr,lag labor,lag gdp_deflator,year1961
60,Afghanistan,2021,426.229401,-22.929194,3.807033,5.58000,4.437872,47.925290,120.623260,553.036131,-5.364666,3.807033,11.71000,4.494553,42.609000,120.050106,0
61,Afghanistan,2022,426.229401,-22.929194,3.807033,6.76000,4.395300,47.925290,84.487488,426.229401,-22.929194,3.807033,5.58000,4.437872,47.925290,120.623260,0
62,Albania,1961,1740.505843,3.648649,13.141789,12.37625,10.223784,66.439813,60.842911,1740.505843,3.648649,13.141789,12.37625,10.223784,66.439813,60.842911,1
63,Albania,1962,1740.505843,3.648649,13.141789,12.37625,10.210841,66.439813,60.842911,1740.505843,3.648649,13.141789,12.37625,10.223784,66.439813,60.842911,0


In [2]:
data.to_csv(r"D:\mas\datasets\World bank\WDI_no_nan.csv")

In [41]:
# 18:27 Dec 12, 2024
# 9:45 Nov 05, 2024
# unit tests for all variables

from statsmodels.tsa.stattools import adfuller

def perform_adf_test(series):
    """
    Perform Augmented Dickey-Fuller test on a single time series.
    Returns the test statistic, p-value, and critical values.
    """
    try:
        result = adfuller(series.dropna())
        return {
            'adf_stat': result[0],
            'p_value': result[1],
            'critical_values': result[4]
        }
    except:
        return {
            'adf_stat': None,
            'p_value': None,
            'critical_values': None
        }

def test_stationarity(data):
    """
    Conduct ADF unit root tests on all numeric columns in the DataFrame.
    Returns a DataFrame with the test results.
    """
    results = []
    for col in data.select_dtypes(include=['number']):
        if col in ['Country Name', 'Year']:
            continue
        test_result = perform_adf_test(data[col])
        results.append({
            'Variable': col,
            'ADF_Stat': test_result['adf_stat'],
            'P_Value': test_result['p_value'],
            'Critical_1%': test_result['critical_values']['1%'],
            'Critical_5%': test_result['critical_values']['5%'],
            'Critical_10%': test_result['critical_values']['10%'],
            'Is_Stationary': test_result['p_value'] < 0.05 if test_result['p_value'] is not None else None
        })
    return pd.DataFrame(results)
unit_test_result = test_stationarity(data)
print(test_stationarity(data).iloc[:20,:])

            Variable   ADF_Stat       P_Value  Critical_1%  Critical_5%  \
0             GDP pc -12.485282  3.028285e-23    -3.430837    -2.861755   
1           ln_GDPpc -18.837641  0.000000e+00    -3.430837    -2.861755   
2                tax -10.630340  5.234217e-19    -3.430837    -2.861755   
3              unemp -12.160025  1.501063e-22    -3.430838    -2.861756   
4             age_dr -13.952221  4.679685e-26    -3.430836    -2.861755   
5              labor -11.544471  3.568741e-21    -3.430837    -2.861755   
6       gdp_deflator -48.423841  0.000000e+00    -3.430836    -2.861755   
7         lag GDP pc -12.192787  1.274444e-22    -3.430837    -2.861755   
8       lag ln_GDPpc -18.473389  2.138295e-30    -3.430837    -2.861755   
9            lag tax -10.600074  6.203841e-19    -3.430837    -2.861755   
10         lag unemp -12.090641  2.126592e-22    -3.430838    -2.861756   
11        lag age_dr -13.933488  5.039884e-26    -3.430836    -2.861755   
12         lag labor -11.

In [43]:
# 10:33 Nov 05, 2024
# save the output bar chart of unit root tests to a png file

import os
import matplotlib.pyplot as plt
import pandas as pd

def save_stationarity_results(data, output_path):
    # Run the stationarity analysis
    #summary_df, overall_summary, detailed_results = test_stationarity(data)

    # Create the output directory if it doesn't exist
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

    # Save the first 20 rows as a PNG image
    plt.figure(figsize=(12, 8))
    #summary_df.iloc[:20, :].plot(kind='bar')
    unit_test_result.iloc[:20,:].plot(kind='bar', x='Variable')
    plt.xticks(rotation=90)
    plt.title('Stationarity Test Results (First 20 Rows)')
    plt.xlabel('Variable')
    plt.ylabel('Value')
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

    print(f"Stationarity test results saved to: {output_path}")

# Example usage
save_stationarity_results(data, r"D:\mas\code\stationarity_results.png")

Stationarity test results saved to: D:\mas\code\stationarity_results.png


<Figure size 1200x600 with 0 Axes>

In [43]:
# 18:32 Dec 12, 2024
# 11:04 Nov 05, 2024
# Granger Causality Test

import statsmodels.api as sm
from statsmodels.tsa.stattools import grangercausalitytests

grangercausalitytests(data[['age_dr', 'unemp']], maxlag=list(range(1,11)))


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.6052  , p=0.4366  , df_denom=13450, df_num=1
ssr based chi2 test:   chi2=0.6053  , p=0.4366  , df=1
likelihood ratio test: chi2=0.6053  , p=0.4366  , df=1
parameter F test:         F=0.6052  , p=0.4366  , df_denom=13450, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.8262  , p=0.1611  , df_denom=13447, df_num=2
ssr based chi2 test:   chi2=3.6538  , p=0.1609  , df=2
likelihood ratio test: chi2=3.6533  , p=0.1609  , df=2
parameter F test:         F=1.8262  , p=0.1611  , df_denom=13447, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=3.6643  , p=0.0118  , df_denom=13444, df_num=3
ssr based chi2 test:   chi2=10.9987 , p=0.0117  , df=3
likelihood ratio test: chi2=10.9942 , p=0.0118  , df=3
parameter F test:         F=3.6643  , p=0.0118  , df_denom=13444, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=7.0446  

{1: ({'ssr_ftest': (0.6051735390298438, 0.43662455069223705, 13450.0, 1),
   'ssr_chi2test': (0.6053085219753522, 0.4365597226282264, 1),
   'lrtest': (0.6052949046570575, 0.43656488175388763, 1),
   'params_ftest': (0.6051735390272031, 0.4366245506939346, 13450.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (1.8262350290896074, 0.16105859223238406, 13447.0, 2),
   'ssr_chi2test': (3.6538281566614708, 0.16090935606848744, 2),
   'lrtest': (3.6533320206435747, 0.16094927748347668, 2),
   'params_ftest': (1.8262350290864728, 0.16105859223297694, 13447.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (3.6643254568349235, 0.011786086913637658, 13444.0, 3),
   'ssr_chi2test': (10.998700175517678, 0.011732906297951604, 3),
   'lrtest': (10.994205881353992, 0.011757247863731366, 3),
   'params_ftest': (3.664325456833904, 0.011786086913654596, 13444.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.]

In [63]:
# 06am 12 Feb 2025
import pandas as pd
path = r"D:\mas\datasets\World bank\Raw data life expectancy at birth.csv"
df = pd.read_csv(path, header=None)

# remove the first 4 rows
df = pd.DataFrame(df.values[4:])

# drop the 1st and 3rd columns
df.drop([1,3], axis=1, inplace=True)

# set year as integer rather than float
df.iloc[0, 2:] = df.iloc[0, 2:].astype(int)

# set first row as the header
df.columns = df.iloc[0]

df = df[1:].reset_index(drop=True)  # Drop the first row and reset index

# select only data['Country Name'] from df['Country Name']
filtered_df = df[df['Country Name'].apply(lambda x: any(sub in x for sub in data['Country Name']))]

# reorder the dataframe as the columns in data in cell 1
filtered_df.sort_values(by = 'Country Name', inplace=True)

# reset the index
filtered_df.reset_index(drop=True, inplace=True)

# drop the column of year 2023
filtered_df.drop(2023, axis=1, inplace=True)
filtered_df.head()


,Country Name,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Afghanistan,"Life expectancy at birth, total (years)",32.535,33.068,33.547,34.016,34.494,34.953,35.453,35.924,...,62.417,62.545,62.659,63.136,63.016,63.081,63.565,62.575,61.982,62.879
1,Albania,"Life expectancy at birth, total (years)",54.439,55.634,56.671,57.844,58.983,60.019,60.998,61.972,...,78.123,78.407,78.644,78.86,79.047,79.184,79.282,76.989,76.463,76.833
2,Algeria,"Life expectancy at birth, total (years)",40.532,40.516,39.323,42.908,42.788,42.714,42.543,42.568,...,74.615,75.11,75.622,75.732,75.743,76.066,76.474,74.453,76.377,77.129
3,American Samoa,"Life expectancy at birth, total (years)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,"Life expectancy at birth, total (years)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Country Name,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Afghanistan,"Life expectancy at birth, total (years)",32.535,33.068,33.547,34.016,34.494,34.953,35.453,35.924,...,62.545,62.659,63.136,63.016,63.081,63.565,62.575,61.982,62.879,NaN
1,Albania,"Life expectancy at birth, total (years)",54.439,55.634,56.671,57.844,58.983,60.019,60.998,61.972,...,78.407,78.644,78.86,79.047,79.184,79.282,76.989,76.463,76.833,NaN
2,Algeria,"Life expectancy at birth, total (years)",40.532,40.516,39.323,42.908,42.788,42.714,42.543,42.568,...,75.11,75.622,75.732,75.743,76.066,76.474,74.453,76.377,77.129,NaN
3,American Samoa,"Life expectancy at birth, total (years)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,"Life expectancy at birth, total (years)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
data['Country Name'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'American Samoa', 'Andorra',
       'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahamas, The', 'Bahrain',
       'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin',
       'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina',
       'Botswana', 'Brazil', 'British Virgin Islands',
       'Brunei Darussalam', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Cayman Islands',
       'Central African Republic', 'Chad', 'Channel Islands', 'Chile',
       'China', 'Colombia', 'Comoros', 'Congo, Dem. Rep.', 'Congo, Rep.',
       'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Curacao',
       'Cyprus', 'Czechia', 'Denmark', 'Djibouti', 'Dominica',
       'Dominican Republic', 'Ecuador', 'Egypt, Arab Rep.', 'El Salvador',
       'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Faroe Islands',